# Library

In [1]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import cv2
import os

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
# faster rcnn model이 포함된 library
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from torch.utils.data import DataLoader, Dataset
import pandas as pd
from tqdm import tqdm

from adamp import AdamP

# Data Related
- Dataset

In [2]:
class CustomDataset(Dataset):
    '''
    Args:
       annotation: Annotation 파일
       data_dir: data가 존재하는 폴더 경로
       transforms: data transform (resize, crop, Totensor, etc,,,)
    '''

    def __init__(self, annotation, data_dir, transforms=None):
        super().__init__()
        self.data_dir = data_dir
        # coco annotation 불러오기 (coco API)
        self.coco = COCO(annotation)
        self.predictions = {
            "images": self.coco.dataset["images"].copy(),
            "categories": self.coco.dataset["categories"].copy(),
            "annotations": None
        }
        self.transforms = transforms

    def __getitem__(self, index: int):  #index 기준으로 image 가져옴
        
        image_id = self.coco.getImgIds(imgIds=index)

        image_info = self.coco.loadImgs(image_id)[0]
        
        # get images
        image = cv2.imread(os.path.join(self.data_dir, image_info['file_name']))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        # get annotations
        ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])
        anns = self.coco.loadAnns(ann_ids)  # id별 annotation 정보 가져옴
        
        # boxex (x_min, y_min, x_max, y_max)
        boxes = np.array([x['bbox'] for x in anns])
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]  # x_max = x_min + Width
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]  # y_max = y_min + height
        
        # get labels
        labels = np.array([x['category_id'] for x in anns])
        labels = torch.as_tensor(labels, dtype=torch.int64)
        
        # ETC, may not be used here
        areas = np.array([x['area'] for x in anns])
        areas = torch.as_tensor(areas, dtype=torch.float32)
                                
        is_crowds = np.array([x['iscrowd'] for x in anns])
        is_crowds = torch.as_tensor(is_crowds, dtype=torch.int64)
                                
        segmentation = np.array([x['segmentation'] for x in anns], dtype=object)

        # Target 구성
        target = {'boxes': boxes, 'labels': labels, 'image_id': torch.tensor([index]), 'area': areas,
                  'iscrowd': is_crowds}

        # transform, 위에 구현된 방식대로 정보들이 transform 되어 반환된다.
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            target['boxes'] = torch.tensor(sample['bboxes'], dtype=torch.float32)

        return image, target, image_id
    
    def __len__(self) -> int:
        return len(self.coco.getImgIds())

### Transforms

In [3]:
def get_train_transform():
    return A.Compose([
        A.Resize(512, 512),
        A.Flip(p=0.5),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})  # bbox 정보도 함께 전달해주어야한다.


def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


# Train Related
- average Meter

In [4]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0


def collate_fn(batch):
    return tuple(zip(*batch))


### Train functions

In [5]:
def train_fn(num_epochs, train_data_loader, optimizer, model, device):
    best_loss = 1000
    loss_hist = Averager()   # 위에서 정의한 Average() loss 기록하는 함수 불러오고
    
    # each epoch
    for epoch in range(num_epochs):
        loss_hist.reset()

        # each batch
        for images, targets, image_ids in tqdm(train_data_loader): 

            # gpu 계산을 위해 image.to(device)
            images = list(image.float().to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            # calculate loss
            loss_dict = model(images, targets)

            losses = sum(loss for loss in loss_dict.values())
            loss_value = losses.item()
            loss_hist.send(loss_value)

            #  optimization & backward
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()

        print(f"Epoch #{epoch+1} loss: {loss_hist.value}")
        # save models
        if loss_hist.value < best_loss:
            torch.save(model.state_dict(), f'faster_rcnn/faster_rcnn.pth')
            best_loss = loss_hist.value

In [6]:
def main():
    annotation = '../../input/data/train.json'
    data_dir = '../../input/data'
    train_dataset = CustomDataset(annotation, data_dir, get_train_transform())

    train_data_loader = DataLoader(
        train_dataset,
        batch_size=16,
        shuffle=False,
        num_workers=4,
        collate_fn=collate_fn
    )

    # get device
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print(device)
    
    # load model
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)  # fpn이 포함된 backbone이resnet인 Faster RcNN 모델 호출

    num_classes = 11   

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    model.to(device)
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

    # change here for further training
    num_epochs = 12

    # trainer
    train_fn(num_epochs, train_data_loader, optimizer, model, device)

In [8]:
if __name__ == '__main__':
    main()

loading annotations into memory...
Done (t=3.46s)
creating index...
index created!
cuda


100%|██████████| 164/164 [04:48<00:00,  1.76s/it]


Epoch #1 loss: 0.925422649739719


100%|██████████| 164/164 [04:48<00:00,  1.76s/it]


Epoch #2 loss: 0.7306202111266008


...

Epoch #10 loss: 0.5383904589476382


100%|██████████| 164/164 [04:48<00:00,  1.76s/it]


Epoch #11 loss: 0.5242781826272244


100%|██████████| 164/164 [04:48<00:00,  1.76s/it]


Epoch #12 loss: 0.5134382147977992


In [ ]:
!pip install adamp